In [6]:
from sklearn.datasets import make_regression
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import cross_val_score
plt.rcParams['figure.figsize'] = [4, 3] 

### **GRADIENT DESCENT :**
* is a first-order iterative optimization algorithm for finding local minimum of a desirable funcn by taking repeated steps in opposite direcn of gradient / slope of funcn at current point.
* As we start moving closer to the soln we take smaller jumps because of decr in slope , Also it ensured we never miss our soln.

**WHEN TO STOP IN GRADIENT DESCENT :**
* when diff abs(bnew - bold) < 0.0001 .
* When no. of epochs completed.

**CALCULATING REGRESSION COEFFICIENT , INTERCEPT & R2 SCORE BY INBUILT FUNCTION IN ADVANCE :**

In [7]:
from sklearn.datasets import load_diabetes
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

X,y = load_diabetes(return_X_y=True)
print(X.shape)
print(y.shape)

(442, 10)
(442,)


In [8]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=2)
reg = LinearRegression()
reg.fit(X_train,y_train)
print(reg.coef_)
print(reg.intercept_)

[  -9.15865318 -205.45432163  516.69374454  340.61999905 -895.5520019
  561.22067904  153.89310954  126.73139688  861.12700152   52.42112238]
151.88331005254167


In [9]:
y_pred = reg.predict(X_test)
print(r2_score(y_test,y_pred))
print(X_train.shape)

0.4399338661568969
(353, 10)


### **COMPARING DIFFERENT GRADIENT DESCENT BY IMPLEMENTING CODE FOR EACH GRADIENT FROM SCRATCH :**
### **1) BATCH GRADIENT DESCENT :**

**BATCH GRADIENT DESCENT CODE FOR MULTIPLE LINEAR REGRESSION :**

In [10]:
class GDRegressor:
    
    def __init__(self,learning_rate=0.01,epochs=100):
        self.coef_ = None
        self.intercept_ = None
        self.lr = learning_rate
        self.epochs = epochs
        
    def fit(self,X_train,y_train):
        # init your coefs
        self.intercept_ = 0
        self.coef_ = np.ones(X_train.shape[1])
        
        for i in range(self.epochs):
            y_hat = np.dot(X_train,self.coef_) + self.intercept_
            intercept_der = -2 * np.mean(y_train - y_hat)
            self.intercept_ = self.intercept_ - (self.lr * intercept_der)
            
            coef_der = -2 * np.dot((y_train - y_hat).T,X_train)/X_train.shape[0]
            self.coef_ = self.coef_ - (self.lr * coef_der)
            
        print(self.intercept_,self.coef_)
    
    def predict(self,X_test):
        return np.dot(X_test,self.coef_) + self.intercept_

In [11]:
gdr = GDRegressor(epochs=1000,learning_rate=0.1)
gdr.fit(X_train,y_train)
y_pred = gdr.predict(X_test)    
print(r2_score(y_test,y_pred))

151.94042847773682 [  62.27835432  -24.14017912  262.40285385  192.20751489   39.48809013
   10.26886323 -142.50597903  124.33312557  244.33510843  119.34350233]
0.3971698388048742


**BATCH GRADIENT DESCENT REQUIRE : LEARNING RATE = 0.1 , EPOCHS = 1000**

### **2) STOCHASTIC GRADIENT DESCENT :**


In [32]:
class SGDRegressor:
    
    def __init__(self,learning_rate=0.01,epochs=100):
        
        self.coef_ = None
        self.intercept_ = None
        self.lr = learning_rate
        self.epochs = epochs
        
    def fit(self,X_train,y_train):
        # init your coefs
        self.intercept_ = 0
        self.coef_ = np.ones(X_train.shape[1])
        
        for i in range(self.epochs):
            for j in range(X_train.shape[0]):
                idx = np.random.randint(0,X_train.shape[0])
                
                y_hat = np.dot(X_train[idx],self.coef_) + self.intercept_
                
                intercept_der = -2 * (y_train[idx] - y_hat)
                self.intercept_ = self.intercept_ - (self.lr * intercept_der)
                
                coef_der = -2 * np.dot((y_train[idx] - y_hat),X_train[idx])
                self.coef_ = self.coef_ - (self.lr * coef_der)
        
        print(self.intercept_,self.coef_)
    
    def predict(self,X_test):
        return np.dot(X_test,self.coef_) + self.intercept_

In [33]:
sgd = SGDRegressor(learning_rate=0.03,epochs=45)

In [34]:
sgd.fit(X_train,y_train)
y_pred = sgd.predict(X_test)
print(r2_score(y_test,y_pred))


155.080450921828 [  15.13924218 -167.3317367   485.01213048  317.83335863  -29.52986851
 -108.86882476 -191.84356966   98.73319661  456.11120832  113.8917772 ]
0.4506556022718634


**STOCHASTIC GRADIENT REQUIRE : LEARNING RATE = 0.03 , EPOCHS = 45**

### **3) MINI BATCH GRADIENT DESCENT :**


In [15]:
import random

class MBGDRegressor:
    
    def __init__(self,batch_size,learning_rate=0.01,epochs=100):
        
        self.coef_ = None
        self.intercept_ = None
        self.lr = learning_rate
        self.epochs = epochs
        self.batch_size = batch_size
        
    def fit(self,X_train,y_train):
        # init your coefs
        self.intercept_ = 0
        self.coef_ = np.ones(X_train.shape[1])
        
        for i in range(self.epochs):
            
            for j in range(int(X_train.shape[0]/self.batch_size)):
                
                idx = random.sample(range(X_train.shape[0]),self.batch_size)
                
                y_hat = np.dot(X_train[idx],self.coef_) + self.intercept_
                #print("Shape of y_hat",y_hat.shape)
                intercept_der = -2 * np.mean(y_train[idx] - y_hat)
                self.intercept_ = self.intercept_ - (self.lr * intercept_der)

                coef_der = -2 * np.dot((y_train[idx] - y_hat),X_train[idx])
                self.coef_ = self.coef_ - (self.lr * coef_der)
        
        print(self.intercept_,self.coef_)
    
    def predict(self,X_test):
        return np.dot(X_test,self.coef_) + self.intercept_

In [30]:
mbr = MBGDRegressor(batch_size=int(X_train.shape[0]/50),learning_rate=0.02,epochs=121)

In [31]:
mbr.fit(X_train,y_train)
y_pred = mbr.predict(X_test)
r2_score(y_test,y_pred)

151.50157075935732 [  -3.4638565  -205.69831838  528.40322432  331.21778334  -56.00958731
 -128.74485923 -187.88878883   87.18175444  493.95250781   79.27865836]


0.45047212624095045

**MINI BATCH GRADIENT REQUIRE : LEARNING RATE = 0.02 , EPOCHS = 121**